In [1]:
import os
import os.path as osp

import numpy as np

import open3d as o3d

import utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd_path = "data/kinth/pointcloud/1720509720.141261339.pcd"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = pcd.voxel_down_sample(voxel_size=0.01)
pcd = pcd.remove_radius_outlier(nb_points=25, radius=0.2)[0] # (pcd, new indexed from old)

points = np.asarray(pcd.points)
points = points - points.mean(axis=0)
print("max, min:", points.max(axis=0), points.min(axis=0))
print(f"before voxelized: {len(points)}")
points, _, _, _ = utils.voxel_downsample(points, 0.02, use_avg=False)
print("max, min:", points.max(axis=0), points.min(axis=0))
print(f"after voxelized: {len(points)}")

max, min: [4.71142875 3.22041954 1.96897121] [-3.35557116 -4.00158058 -1.04102878]
before voxelized: 12234
max, min: [8.07 7.23 3.01] [0.01 0.01 0.01]
after voxelized: 7677


# height filtration

In [3]:
# pivot = utils.get_average_pivot(points[:, 2])
# points = points[points[:, 2] < pivot * 1.75]

# covariance rad feature

In [4]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pickle

search_radius = 2.5
feat_rad_list = None
feat_rad_list_dump_file = f"data/kinth/pickle/feat_rad_{search_radius:.2f}_{osp.basename(pcd_path).split('.')[0]}.pkl"

# if osp.exists(feat_rad_list_dump_file):
#     with open(feat_rad_list_dump_file, "rb") as f:
#         feat_rad_list = pickle.load(f)
# else:
    # 多进程处理子空间坐标几何特征计算
num_worker = 4
batch_size = (len(points) - 1) // num_worker + 1 # round up
feat_rad_list_con = [None] * num_worker

with ProcessPoolExecutor(max_workers=num_worker) as executor:
    future_dict = {
        executor.submit(
            utils.eigval_radius,
            points.copy(),
            (i * batch_size, min((i + 1) * batch_size, len(points))),
            search_radius
        ) : i for i in range(num_worker)
    }
    for future in as_completed(future_dict):
        feat_list, neighbor_num_record = future.result()
        feat_rad_list_con[future_dict[future]] = feat_list
        print(f"missing:{(np.array(neighbor_num_record) < 3).astype(np.int32).sum()}")
feat_rad_list = np.concatenate(feat_rad_list_con, axis=0)
    
    # if not osp.exists(osp.dirname(feat_rad_list_dump_file)):
    #     os.makedirs(osp.dirname(feat_rad_list_dump_file), exist_ok=True)
    # with open(feat_rad_list_dump_file, "wb") as f:
    #     pickle.dump(feat_rad_list, f)

eigval progress: 100%|████████████████████████████████████████| 1920/1920 [00:00<00:00, 3208.85it/s]


missing:0
missing:0
missing:0


eigval progress: 100%|████████████████████████████████████████| 1920/1920 [00:00<00:00, 2984.00it/s]


missing:0


In [5]:
print(feat_rad_list)
feat_rad_color = feat_rad_list
feat_rad_color = feat_rad_color / (feat_rad_color.sum(axis=1).reshape(-1, 1) + 1e-9)
feat_rad_color = (feat_rad_color * 255.0).astype(np.int32)
print(feat_rad_color)


[[0.56037027 0.61389065 0.83025485]
 [0.40818853 0.69014165 0.81662227]
 [0.67753729 0.59564836 0.86961167]
 ...
 [0.40037955 0.95581884 0.97350807]
 [0.40037955 0.95581884 0.97350807]
 [0.40037955 0.95581884 0.97350807]]
[[ 71  78 105]
 [ 54  91 108]
 [ 80  70 103]
 ...
 [ 43 104 106]
 [ 43 104 106]
 [ 43 104 106]]


In [6]:
utils.npy2ply(points, feat_rad_color, "data/output/sample_rad.ply")

In [7]:
# empty slot for staged run

# covariance vtl feature

In [8]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pickle

search_radius = 2.0
feat_vtl_list = None
feat_vtl_list_dump_file = f"data/kinth/pickle/feat_vtl_{search_radius:.2f}_{osp.basename(pcd_path).split('.')[0]}.pkl"

# if osp.exists(feat_vtl_list_dump_file):
#     with open(feat_vtl_list_dump_file, "rb") as f:
#         feat_vtl_list = pickle.load(f)
# else:
    # 多进程处理子空间坐标几何特征计算
num_worker = 4
batch_size = (len(points) - 1) // num_worker + 1 # round up
feat_vtl_list_con = [None] * num_worker

with ProcessPoolExecutor(max_workers=num_worker) as executor:
    future_dict = {
        executor.submit(
            utils.eigval_vertic,
            points,
            (i * batch_size, min((i + 1) * batch_size, len(points))),
            search_radius
        ) : i for i in range(num_worker)
    }
    for future in as_completed(future_dict):
        feat_list, neighbor_num_record = future.result()
        print(f"missing: {(np.array(neighbor_num_record) < 3).astype(np.int32).sum()}")
        feat_vtl_list_con[future_dict[future]] = feat_list
feat_vtl_list = np.concatenate(feat_vtl_list_con, axis=0)
    
    # if not osp.exists(osp.dirname(feat_vtl_list_dump_file)):
    #     os.makedirs(osp.dirname(feat_vtl_list_dump_file), exist_ok=True)
    # with open(feat_vtl_list_dump_file, "wb") as f:
    #     pickle.dump(feat_vtl_list, f)

eigval progress: 100%|████████████████████████████████████████| 1920/1920 [00:00<00:00, 3673.94it/s]


missing: 0
missing: 0
missing: 0
missing: 0


In [9]:
print(feat_vtl_list)
feat_vtl_color = feat_vtl_list
feat_vtl_color = feat_vtl_color / (feat_vtl_color.sum(axis=1).reshape(-1, 1) + 1e-9)
feat_vtl_color = (feat_vtl_color * 255.0).astype(np.int32)
print(feat_vtl_color)

[[0.80787677 0.0622222  0.58605761]
 [0.80787677 0.0622222  0.58605761]
 [0.80727165 0.06878825 0.58615755]
 ...
 [0.75716611 0.38267659 0.52939409]
 [0.75941932 0.36428799 0.53905154]
 [0.76012291 0.38032145 0.52684795]]
[[141  10 102]
 [141  10 102]
 [140  11 102]
 ...
 [115  58  80]
 [116  55  82]
 [116  58  80]]


In [10]:
utils.npy2ply(points, feat_vtl_color, "data/output/sample_vtl.ply")

# cluster average feature

In [11]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pickle

search_radius = 0.75
feat_avg_list = None
feat_avg_list_dump_file = f"data/kinth/pickle/feat_avg_{search_radius:.2f}_{osp.basename(pcd_path).split('.')[0]}.pkl"

feat_avg_list, neighbor_num_record = utils.avgvec_vertic(points, (0, len(points)), search_radius)
feat_avg_list = np.array(feat_avg_list)
print((np.array(neighbor_num_record) <= 1).astype(np.int32).sum())
# num_worker = 4
# batch_size = (len(points) - 1) // num_worker + 1 # round up
# feat_avg_list_con = [None] * num_worker

# with ProcessPoolExecutor(max_workers=num_worker) as executor:
#     future_dict = {
#         executor.submit(
#             utils.avgvec_vertic,
#             points,
#             (i * batch_size, min((i + 1) * batch_size, len(points))),
#             search_radius
#         ) : i for i in range(num_worker)
#     }
#     for future in as_completed(future_dict):
#         feat_list, neighbor_num_record = future.result()
#         print(f"missing: {(np.array(neighbor_num_record) < 3).astype(np.int32).sum()}")
#         feat_avg_list_con[future_dict[future]] = feat_list
# feat_avg_list = np.concatenate(feat_avg_list_con, axis=0)

print(feat_avg_list)
feat_avg_color = np.ones((len(points), 3))
feat_avg_color = (feat_avg_color * feat_avg_list * 255.0).astype(np.int32)
print(feat_avg_color)

eigval progress:   0%|                                                     | 0/7677 [00:00<?, ?it/s]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
utils.npy2ply(points, feat_avg_color, "data/output/sample_avg.ply")